# Deep Learning without Spark

In [1]:
import numpy as np 
import pandas as pd
from scipy.ndimage import variance, mean, median, standard_deviation, sum, maximum
from skimage import io
from skimage.color import rgb2gray
from sklearn import preprocessing
from skimage.filters import laplace, sobel, scharr, prewitt
from skimage.transform import resize
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.svm import SVC
import seaborn as sb
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten
import cv2
import os

Using TensorFlow backend.


In [2]:
# load model without classifier layers
model = InceptionV3(include_top=False, input_shape=(299, 299, 3))

In [3]:
path_macro = "/Users/vyasaa96/CS631/Project/Images/macroblocked/"
path_pristine = "/Users/vyasaa96/CS631/Project/Images/pristine/"
macro_imgs = os.listdir(path_macro)
macro_imgs1 = list(map(lambda x:path_macro+str(x),macro_imgs))
pristine_imgs = os.listdir(path_pristine)
pristine_imgs1 = list(map(lambda x:path_pristine+str(x),pristine_imgs))
macro_imgs1

['/Users/vyasaa96/CS631/Project/Images/macroblocked/3975.png',
 '/Users/vyasaa96/CS631/Project/Images/macroblocked/348.png',
 '/Users/vyasaa96/CS631/Project/Images/macroblocked/1804.png',
 '/Users/vyasaa96/CS631/Project/Images/macroblocked/4968.png',
 '/Users/vyasaa96/CS631/Project/Images/macroblocked/1810.png',
 '/Users/vyasaa96/CS631/Project/Images/macroblocked/3961.png',
 '/Users/vyasaa96/CS631/Project/Images/macroblocked/1186.png',
 '/Users/vyasaa96/CS631/Project/Images/macroblocked/412.png',
 '/Users/vyasaa96/CS631/Project/Images/macroblocked/3949.png',
 '/Users/vyasaa96/CS631/Project/Images/macroblocked/3791.png',
 '/Users/vyasaa96/CS631/Project/Images/macroblocked/4798.png',
 '/Users/vyasaa96/CS631/Project/Images/macroblocked/4940.png',
 '/Users/vyasaa96/CS631/Project/Images/macroblocked/374.png',
 '/Users/vyasaa96/CS631/Project/Images/macroblocked/1838.png',
 '/Users/vyasaa96/CS631/Project/Images/macroblocked/360.png',
 '/Users/vyasaa96/CS631/Project/Images/macroblocked/4954.pn

In [4]:
#label the data
macro_label = [1]*len(macro_imgs1)
pristine_label = [0]*len(pristine_imgs1)

In [5]:
#separate data from labels
macro_df = pd.DataFrame(macro_imgs1,columns=['Path'])
macro_df['Label'] = macro_label
pristine_df = pd.DataFrame(pristine_imgs1,columns=['Path'])
pristine_df['Label'] = pristine_label

In [6]:
#join and shuffle macroblock and pristine images
all_images = pd.concat([macro_df,pristine_df])
all_images = all_images.sample(frac=1).reset_index(drop=True)

In [7]:
temp = all_images.iloc[0:1000,:]#varying the data size for experimentation

In [8]:
#batch processing of image data directly from the directories
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory('/Users/vyasaa96/CS631/Project/Images_1000',
                                                 target_size=(299,299),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 4420 images belonging to 2 classes.


In [9]:
# add new classifier layers
flat1 = GlobalAveragePooling2D()(model.output)
# class1 = Dense(1024, activation='relu')(flat1)
output = Dense(2, activation='softmax')(flat1)
# define new model
model = Model(inputs=model.inputs, outputs=output)

In [11]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
# Adam optimizer
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=1)
# model.save("inception.h5")